In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import string
import numpy as np
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
def google_search(x,n):
    driver = webdriver.Chrome('chromedriver')
    driver.get(f'https://www.google.com/search?q={x}')
    next_page_times = n
    http=[]
    for _page in range(next_page_times):
    
        soup = BeautifulSoup(driver.page_source,'html.parser')
        items=soup.find_all('div', attrs={'class':'yuRUbf'})

        for i in items:
            # 網址
            h=i.find('a')['href']
            if 'cdc.gov.tw' in h :
                pass
            else:
                http.append(h)
        # Wait
        time.sleep(random.uniform(1, 5))

        # Turn to the next page
        try:
            driver.find_element_by_link_text('下一頁').click()
        except:
            print('Search Early Stopping.')
            driver.close()
            exit()
    driver.close()
    return http

In [4]:
def the_economist(x):
    eco=BeautifulSoup(x.text, 'html.parser')
    #標題
    try:
        titles=eco.find('title').text.split('|')[0]
    except:
        titles='None'
    #內容
    try:
        content=''
        for i in eco.find_all('p',attrs={'class':"article__body-text"}):
            content+=i.text
    except:
        content='None'
    #字數
    words=len(content.split(' '))
    #分類
    try:
        sention=eco.find('span',attrs={'class':"article__section-headline"}).find('a').text
    except:
        sention='None'
    #日期
    try:
        date=eco.find('meta',attrs={'itemprop':"datePublished"})['content'][0:10]
    except:
        date='None'
    #作者
    for i in eco.find_all('link',attrs={'rel':'preconnect'}):
        try:
            author=i['crossorigin']
        except:
            pass
    #圖片數
    figure=[]
    for i in eco.find_all('div',attrs={'itemprop':'image'}):
        try :
            if i['class'][0]=="related-article__image":
                pass
            else:
                figure.append(i)
        except:
            figure.append(i)
    n_p=len(figure)
    #音檔數
    m_p=len(eco.find_all('div',attrs={'class':'article-audio-player'}))
    if m_p !=0:
        all_p=f'{n_p}（+{m_p}音檔）'
    else:
        all_p=n_p
    if 'vaccin' in content :
        return ['','',author,'',sention,'',date,titles,content,'','',words,'',all_p,'']
    else:
        return['','','','','','','','','','','','','','','']

In [5]:
def web_crawl(x,n):
    http=google_search(x,n)
    economist=[]
    for i in range(0,len(http)):
        economist.append(requests.get(http[i]))
        time.sleep(random.uniform(1, 5))
    eco_result=list(map(the_economist,economist))
    eco_df=pd.DataFrame(eco_result)
    eco_df=eco_df.rename(index=str,columns={0:'',1:'',2:'作者',3:'',4:'記者路線',5:'',6:'日期',7:'標題',8:'內文',9:'',10:'',11:'字數',12:'',13:"附圖（幀）",14:''})
    eco_df['網址']=http
    eco_df=eco_df[eco_df.標題!='']
    return eco_df

In [6]:
df=web_crawl('vaccine after:2021-07-01 before:2021-07-10 site:economist.com',2)

In [7]:
df

,,,作者,,記者路線,,日期,標題,內文,,,字數,,附圖（幀）,,網址
0,,,anonymous,,The Economist explains,,2021-07-07,What does Brazil’s vaccine scandal mean for Ja...,JAIR BOLSONARO ran for president on a promise ...,,,658,,1,,https://www.economist.com/the-economist-explai...
1,,,anonymous,,The Economist explains,,2021-07-02,Why don’t governments make covid-19 vaccines m...,IN 1853 THE British government passed the worl...,,,596,,1,,https://www.economist.com/the-economist-explai...
2,,,anonymous,,Europe,,2021-07-10,Covid-19 deaths in Russia are soaring,FOR THE past year the Kremlin has trumpeted it...,,,947,,2（+1音檔）,,https://www.economist.com/europe/2021/07/10/co...
3,,,anonymous,,Graphic detail,,2021-07-08,Why has America’s vaccination programme slowed...,ON JULY 4TH President Joe Biden stood on the W...,,,642,,1,,https://www.economist.com/graphic-detail/2021/...
4,,,anonymous,,Films,,2021-07-09,Vaccines and variants,This year is shaping up to be the year of the ...,,,144,,1,,https://www.economist.com/films/2021/07/09/vac...
5,,,anonymous,,Briefing,,2021-07-03,The unvaccinated are at risk as evolution acce...,"FOR MUCH of 2020 the covid-19 virus was, in ge...",,,3472,,6（+1音檔）,,https://www.economist.com/briefing/2021/07/03/...
6,,,anonymous,,Leaders,,2021-07-03,The long goodbye to covid-19,"WHEN WILL it end? For a year and a half, covid...",,,1073,,1（+1音檔）,,https://www.economist.com/leaders/2021/07/03/t...
7,,,anonymous,,Britain,,2021-07-03,The anti-lockdown movement is still going strong,JAM FOR FREEDOM pumps out Oasis and Bob Marley...,,,664,,1（+1音檔）,,https://www.economist.com/britain/2021/07/03/t...
8,,,anonymous,,Films,,2021-07-01,Editor’s conversations: Ngozi Okonjo-Iweala,“It’s not about helping Africa or helping low-...,,,211,,1,,https://www.economist.com/films/2021/07/01/edi...
9,,,anonymous,,What If?,,2021-07-03,What if biohackers injected themselves with mR...,"Editor’s note: This year What If?, our annual ...",,,1124,,3（+1音檔）,,https://www.economist.com/what-if/2021/07/03/w...


In [8]:
df.to_excel('經濟學人--7月.xlsx')

In [206]:
r

['https://www.google.com/search?q=vaccines+after:2021-01-01+before:2021-01-31+site:economist.com&sxsrf=ALeKk035V3G3SXvH2GjzYMJ4nYgV6VCP2w:1621444334807&source=lnms&sa=X&ved=2ahUKEwj-l-XsntbwAhUCQd4KHe_7B0YQ_AUoAHoECAEQAg&biw=1280&bih=610&dpr=1.5',
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [209]:
r=[]
r1= requests.get('https://www.google.com/search?q=vaccines+after:2021-01-01+before:2021-01-31+site:economist.com&sxsrf=ALeKk035V3G3SXvH2GjzYMJ4nYgV6VCP2w:1621444334807&source=lnms&sa=X&ved=2ahUKEwj-l-XsntbwAhUCQd4KHe_7B0YQ_AUoAHoECAEQAg&biw=1280&bih=610&dpr=1.5')
r.append(r1)
title=[]
http=[]
for i in range(0,len(r)):
        # 以 BeautifulSoup 解析 HTML 原始碼
    soup = BeautifulSoup(r[0].text, 'html.parser')

        #觀察 HTML 原始碼
        # print(soup.prettify())

          # 以 CSS 的選擇器來抓取 Google 的搜尋結果
    items = soup.select('a[href^="/url"]')

    items= items[:-2]
    for i in items:
            # 標題
        title.append(i.text)
            # 網址
        http.append(i.get('href')[7:].split('&')[0])

In [212]:
title[1]

'The great task | The Economistwww.economist.com › Briefing › Jan 7th 2021 edition'

In [213]:
http[0]

'https://www.economist.com/graphic-detail/2021/01/28/vaccine-nationalism-means-that-poor-countries-will-be-left-behind'

In [214]:
e=requests.get('https://www.airitilibrary.com/Publication/alDetailedMesh?DocID=10288694-200801-x-39-119-146-a')

In [215]:
eco=BeautifulSoup(e.text, 'html.parser')

In [284]:
print(eco.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   window.env = {"ENV":"production"};
  </script>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <title>
   Vaccine nationalism means that poor countries will be left behind | The Economist
  </title>
  <link href="/engassets/ico/favicon.f1ea908894.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/engassets/ico/touch-icon-120x120.f1ea908894.png" rel="apple-touch-icon"/>
  <link href="/engassets/ico/touch-icon-180x180.f1ea908894.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/engassets/ico/touch-icon-152x152.f1ea908894.png" rel="apple-touch-icon" sizes="152x152"/>
  <link href="/engassets/ico/touch-icon-167x167.f1ea908894.png" rel="apple-touch-icon" sizes="167x167"/>
  <link href="/site.webmanifest.json" rel="manifest"/>
  <link href="https://cdn.design-system.economist.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://cdn.design-system.economist.com" rel=

In [217]:
#標題
title=eco.find('title').text.split('|')[0]
title

'Vaccine nationalism means that poor countries will be left behind '

In [218]:
#內容
content=''
for i in eco.find_all('p',attrs={'class':"article__body-text"}):
    content+=i.text

In [219]:
#content

In [220]:
#字數
words=len(content.split(' '))
print(words)

522


In [221]:
#分類
sention=eco.find('span',attrs={'class':"article__section-headline"}).find('a').text
sention

'Graphic detail'

In [222]:
#日期
date=eco.find('meta',attrs={'itemprop':"datePublished"})['content'][0:10]
date

'2021-01-28'

In [268]:
#作者
for i in eco.find_all('link',attrs={'rel':'preconnect'}):
    try:
        author=i['crossorigin']
    except:
        pass
author

'anonymous'

In [224]:
#圖片數
figure=[]
for i in eco.find_all('div',attrs={'itemprop':'image'}):
    try :
        if i['class'][0]=="related-article__image":
            pass
        else:
            figure.append(i)
    except:
        figure.append(i)
n_p=len(figure)
print(n_p)

1


In [225]:
#音檔數
m_p=len(eco.find_all('div',attrs={'class':'article-audio-player'}))

In [226]:
m_p

0